## Chapter 14: Middleware Pipeline

Every request that hits your ASP.NET Core application travels through a **pipeline** of components called **middleware**. Each piece of middleware can inspect, modify, or short‑circuit the request and response. Understanding this pipeline is essential for building secure, performant, and maintainable applications. In this chapter, you’ll learn what middleware is, how to compose a pipeline using `Use`, `Run`, and `Map`, the importance of ordering, and how to write your own custom middleware. By the end, you’ll be able to control the request flow and add cross‑cutting concerns like logging, authentication, and error handling exactly where they belong.

### 14.1 What is Middleware?

Middleware is software that’s assembled into an application pipeline to handle requests and responses. Each component:

- Can choose whether to pass the request to the next component in the pipeline.
- Can perform work before and after the next component.
- Is typically a class with an `InvokeAsync` method (or just a function).

Think of it as a series of layers that the request passes through on its way to your application logic, and the response passes through on its way back.

**Simple example of a pipeline:**

```
Request → Middleware1 → Middleware2 → Endpoint → Middleware2 → Middleware1 → Response
```

The request enters the first middleware, which may do some work, then calls the next middleware, and so on. When the endpoint is reached (e.g., a controller action), the response is generated and then flows back through the pipeline in reverse order.

#### Built‑in Middleware

ASP.NET Core includes many built‑in middleware components:

- **Static Files**: Serves static files (CSS, JavaScript, images) and short‑circuits the pipeline if a file is found.
- **Routing**: Matches the request to an endpoint.
- **Authentication**: Establishes the user’s identity.
- **Authorization**: Checks if the user is allowed to access the endpoint.
- **Exception Handling**: Catches exceptions and returns friendly responses.
- **CORS**: Adds Cross‑Origin Resource Sharing headers.
- **HTTPS Redirection**: Redirects HTTP requests to HTTPS.

You add these to the pipeline in `Program.cs` using extension methods like `app.UseStaticFiles()`, `app.UseRouting()`, etc.

### 14.2 The Request Delegate and `Use()`, `Run()`, `Map()`

The pipeline is built by defining **request delegates**. A request delegate is a function that takes an `HttpContext` and returns a `Task`. You can combine delegates using three main methods: `Use`, `Run`, and `Map`.

#### `Use` – Chaining Middleware

The `Use` method adds a middleware component that can call the next delegate. It takes a `Func<RequestDelegate, RequestDelegate>` or, more commonly, you use a lambda that receives `HttpContext` and a `Func<Task>` representing the next delegate.

```csharp
app.Use(async (context, next) =>
{
    // Do work before the next middleware
    Console.WriteLine("Before next");
    await next(); // Call the next middleware
    // Do work after the next middleware
    Console.WriteLine("After next");
});
```

The `next` delegate may not be called if the middleware decides to short‑circuit the pipeline (e.g., if it returns a response directly).

#### `Run` – Terminal Middleware

`Run` adds a middleware that does **not** call a next delegate; it terminates the pipeline. It takes a `RequestDelegate` (a function that handles the request directly).

```csharp
app.Run(async context =>
{
    await context.Response.WriteAsync("Hello from terminal middleware");
});
```

Any middleware added after `Run` will never be executed because the pipeline ends here.

#### `Map` – Branching the Pipeline

`Map` branches the pipeline based on the request path. If the request path matches the given path, the branch is executed; otherwise, it continues.

```csharp
app.Map("/admin", adminApp =>
{
    adminApp.Use(async (context, next) =>
    {
        // This only runs for paths starting with "/admin"
        await next();
    });
    adminApp.Run(async context =>
    {
        await context.Response.WriteAsync("Admin area");
    });
});
```

You can also use `MapWhen` for more complex conditions based on the `HttpContext`.

```csharp
app.MapWhen(context => context.Request.Query.ContainsKey("special"), appBranch =>
{
    appBranch.Run(async context =>
    {
        await context.Response.WriteAsync("Special branch");
    });
});
```

### 14.3 Ordering of Middleware (Security, Static Files, Routing)

The order in which middleware is added to the pipeline **matters critically**. Each middleware can depend on data from previous ones and can affect subsequent ones. A common ordering pattern for web applications is:

1. **Exception Handling** – Catch exceptions early so you can return a friendly response.
2. **HTTPS Redirection** – Ensure secure connections.
3. **Static Files** – Serve static files quickly; often placed early to short‑circuit for performance.
4. **Routing** – Determine the endpoint.
5. **Authentication** – Establish user identity.
6. **Authorization** – Enforce access rules.
7. **Endpoints** – Execute the actual application logic (e.g., MVC controllers).

In code, this looks like:

```csharp
var app = builder.Build();

if (app.Environment.IsDevelopment())
{
    app.UseDeveloperExceptionPage();
}
else
{
    app.UseExceptionHandler("/Home/Error");
    app.UseHsts();
}

app.UseHttpsRedirection();
app.UseStaticFiles();

app.UseRouting();

app.UseAuthentication();
app.UseAuthorization();

app.MapControllerRoute(
    name: "default",
    pattern: "{controller=Home}/{action=Index}/{id?}");

app.Run();
```

Let's examine why this order works:

- **Exception Handling** must be first so it can catch exceptions from everything else.
- **Static Files** comes before routing because if a static file is requested, we want to serve it immediately without going through authentication, authorization, etc. (unless you need to protect static files, in which case you'd move it later).
- **Routing** must come before Authentication/Authorization because it needs to determine the endpoint so that later middleware know what endpoint is being accessed.
- **Authentication** runs before Authorization because we need to know who the user is before we decide if they're allowed.
- **Authorization** then checks permissions based on the user and the endpoint.
- Finally, the **endpoint** middleware executes the selected action.

If you placed `UseAuthorization` before `UseAuthentication`, authorization would fail because the user would not be known.

### 14.4 Writing Custom Middleware

You often need to add custom logic that runs for every request—for example, logging, request/response timing, modifying headers, or validating API keys. You can write your own middleware as an inline lambda or as a reusable class.

#### Inline Middleware (for simple tasks)

```csharp
app.Use(async (context, next) =>
{
    // Add a custom header to all responses
    context.Response.Headers.Add("X-Server-Name", "MyServer");
    await next();
});
```

#### Class‑based Middleware (for reusability and DI)

Create a class with an `InvokeAsync` method:

```csharp
public class RequestTimingMiddleware
{
    private readonly RequestDelegate _next;
    private readonly ILogger<RequestTimingMiddleware> _logger;

    public RequestTimingMiddleware(RequestDelegate next, ILogger<RequestTimingMiddleware> logger)
    {
        _next = next;
        _logger = logger;
    }

    public async Task InvokeAsync(HttpContext context)
    {
        var start = DateTime.UtcNow;

        // Call the next middleware
        await _next(context);

        var elapsed = DateTime.UtcNow - start;
        _logger.LogInformation("Request {Method} {Path} took {ElapsedMs}ms",
            context.Request.Method, context.Request.Path, elapsed.TotalMilliseconds);
    }
}
```

Then, register it in the pipeline using `UseMiddleware<T>`:

```csharp
app.UseMiddleware<RequestTimingMiddleware>();
```

Or create an extension method for cleaner syntax:

```csharp
public static class RequestTimingMiddlewareExtensions
{
    public static IApplicationBuilder UseRequestTiming(this IApplicationBuilder builder)
    {
        return builder.UseMiddleware<RequestTimingMiddleware>();
    }
}

// In Program.cs
app.UseRequestTiming();
```

#### Middleware with Dependencies

If your middleware needs services from DI (like `ILogger<T>`), you can inject them in the constructor as shown above. However, note that the middleware is instantiated once (singleton‑like). If you need scoped services, inject `IServiceProvider` in the constructor and create a scope in `InvokeAsync`, or inject the scoped service into the `InvokeAsync` method itself (since `InvokeAsync` parameters are populated from DI).

```csharp
public class CustomMiddleware
{
    private readonly RequestDelegate _next;

    public CustomMiddleware(RequestDelegate next)
    {
        _next = next;
    }

    public async Task InvokeAsync(HttpContext context, IMyScopedService scopedService)
    {
        // use scopedService
        await _next(context);
    }
}
```

#### Short‑Circuiting the Pipeline

Middleware can decide not to call `next()`, thereby short‑circuiting the pipeline. This is useful for serving static files, handling errors, or implementing custom authorization.

```csharp
app.Use(async (context, next) =>
{
    if (context.Request.Path == "/ping")
    {
        context.Response.ContentType = "text/plain";
        await context.Response.WriteAsync("pong");
        return; // Don't call next – pipeline ends here
    }
    await next(); // Otherwise continue
});
```

#### Example: API Key Validation Middleware

Let's build a custom middleware that validates an API key for a specific path prefix.

```csharp
public class ApiKeyMiddleware
{
    private readonly RequestDelegate _next;
    private const string API_KEY_HEADER = "X-API-Key";
    private readonly string _validApiKey;

    public ApiKeyMiddleware(RequestDelegate next, IConfiguration configuration)
    {
        _next = next;
        _validApiKey = configuration["ApiKey"]; // Retrieve from configuration
    }

    public async Task InvokeAsync(HttpContext context)
    {
        // Only validate for paths starting with /api
        if (context.Request.Path.StartsWithSegments("/api"))
        {
            if (!context.Request.Headers.TryGetValue(API_KEY_HEADER, out var extractedApiKey))
            {
                context.Response.StatusCode = 401;
                await context.Response.WriteAsync("API Key missing");
                return;
            }

            if (!_validApiKey.Equals(extractedApiKey))
            {
                context.Response.StatusCode = 403;
                await context.Response.WriteAsync("Invalid API Key");
                return;
            }
        }

        await _next(context);
    }
}

// Extension method
public static class ApiKeyMiddlewareExtensions
{
    public static IApplicationBuilder UseApiKey(this IApplicationBuilder builder)
    {
        return builder.UseMiddleware<ApiKeyMiddleware>();
    }
}
```

Register it after `UseRouting` but before `UseAuthorization` (or before endpoints if you want to protect all API endpoints).

### Summary

Middleware is the backbone of request processing in ASP.NET Core. In this chapter, you learned:

- **What middleware is** and how it forms a pipeline.
- How to add middleware using **`Use`, `Run`, and `Map`**.
- The critical **ordering** of middleware and why it matters.
- How to write **custom middleware** classes and inline lambdas.
- How to **short‑circuit** the pipeline and use dependency injection in middleware.

With these skills, you can add cross‑cutting concerns to your application in a clean, reusable way. Middleware is the right place for request logging, header manipulation, API key validation, and many other tasks.

**Exercise:**

1. Write a custom middleware that measures the time taken to process each request and adds a response header `X-Response-Time-ms` with the elapsed milliseconds.
2. Add this middleware to your pipeline and test it.
3. Create a simple middleware that blocks requests from a specific IP address (you can check `context.Connection.RemoteIpAddress`). Return a 403 Forbidden for those IPs.
4. Experiment with the order of middleware: place your timing middleware first and then later; observe the difference in measured time.

In the next chapter, **"Advanced C# for Performance,"** you'll dive into performance‑oriented C# features like `Span<T>`, `Memory<T>`, `ValueTask`, and source generators. These advanced topics will help you write high‑performance code, especially important for large‑scale applications and libraries.